# Collaborative Filtering - Intuition

In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv("data/ratings.csv")

In [3]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## Explicit Prediction Rating

In [78]:
items = pd.read_csv("data/items.csv")
users = pd.read_csv("data/users.csv")
ratings = pd.read_csv("data/ratings.csv")

In [29]:
items.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Small Data

| movie_id | title                 |
| -------: | :-------------------- |
|      1   | Toy Story (1995)      |
|     71   | Lion King, The (1994) |
|     95   | Aladdin (1992)        |
|     50   | Star Wars (1972)      |
|    176   | Aliens (1986)         |
|     82   | Jurassic Park (1993)  |



In [79]:
sample_movie_id = [1, 71, 95, 50, 176, 82]
sample_user_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [80]:
sample_items = items[items.movie_id.isin(sample_movie_id)]
sample_users = users[users.user_id.isin(sample_user_id)]

In [81]:
sample_items.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
49,50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,...,0,0,0,0,0,1,1,0,1,0
70,71,"Lion King, The (1994)",01-Jan-1994,NaN,"http://us.imdb.com/M/title-exact?Lion%20King,%...",0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
81,82,Jurassic Park (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Jurassic%20Pa...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
94,95,Aladdin (1992),01-Jan-1992,NaN,http://us.imdb.com/M/title-exact?Aladdin%20(1992),0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0


In [40]:
#items.query("Action == 1 and SciFi == 1")
#items.query("Children == 1 and Animation == 1")

In [83]:
sample_users

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [87]:
rating_collect = ratings.user_id.isin(sample_user_id) & ratings.movie_id.isin(sample_movie_id)

In [88]:
sample_ratings = ratings[rating_collect]

In [89]:
sample_ratings.head()

,user_id,movie_id,rating,unix_timestamp
1052,2,50,5,888552084
1090,8,50,5,879362124
1333,5,1,4,875635748
3672,6,95,2,883602133
4280,1,82,5,878542589


In [90]:
len(sample_ratings.user_id.unique())

14

In [91]:
pd.crosstab(index=sample_ratings.user_id, columns=sample_ratings.movie_id, values=sample_ratings.rating, aggfunc="mean")

movie_id,1,50,71,82,95,176
user_id,,,,,,
1,5.0,5.0,3.0,5.0,4.0,5.0
2,4.0,5.0,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN
5,4.0,4.0,NaN,NaN,4.0,3.0
6,4.0,4.0,4.0,NaN,2.0,NaN
7,NaN,5.0,5.0,3.0,NaN,3.0
8,NaN,5.0,NaN,5.0,NaN,5.0
9,NaN,5.0,NaN,NaN,NaN,NaN
10,4.0,5.0,NaN,4.0,NaN,4.0


In [93]:
import keras

ModuleNotFoundError: No module named 'keras'

In [ ]:
# 